# Error Handling and Recovery Patterns

This notebook demonstrates robust error handling with the LouieAI notebook interface.

**Topics covered:**
- Handling authentication errors
- Recovering from query failures
- Dealing with missing data
- Error inspection and debugging
- Graceful fallbacks

## 1. Setup with Error Handling

In [ ]:
import os

import pandas as pd

# Demonstration: Handling missing credentials gracefully
try:
    from louieai.notebook import lui

    # Check credentials
    if not os.environ.get('GRAPHISTRY_USERNAME'):
        print("⚠️  No credentials found in environment")
        print("\nTo use this notebook:")
        print("1. Set GRAPHISTRY_USERNAME and "
              "GRAPHISTRY_PASSWORD environment variables")
        print("2. Restart the kernel")
        print("\nExample:")
        print("export GRAPHISTRY_USERNAME=your_username")
        print("export GRAPHISTRY_PASSWORD=your_password")
    else:
        print("✅ LouieAI ready with credentials")
        print(f"User: {os.environ.get('GRAPHISTRY_USERNAME')}")

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\nPlease install louieai: pip install louieai")

## 2. Handling Query Errors

In [ ]:
# Example: Handling various query scenarios
from louieai.notebook.exceptions import (
    AuthenticationError,
    ConnectionError,
    NotebookError,
)


def safe_query(prompt, fallback_action=None):
    """Execute a query with comprehensive error handling."""
    try:
        response = lui(prompt)

        # Check for errors in response
        if lui.has_errors:
            print("⚠️  Query completed with errors:")
            for error in lui.errors:
                print(f"  - {error.get('message', 'Unknown error')}")

        return response

    except AuthenticationError:
        print("❌ Authentication failed")
        print("💡 Check your GRAPHISTRY_USERNAME and "
              "GRAPHISTRY_PASSWORD environment variables")

    except ConnectionError as e:
        print(f"❌ Connection error: {e}")
        print("💡 Check your internet connection and server URL")

    except NotebookError as e:
        print(f"❌ Notebook error: {e}")
        if hasattr(e, 'suggestion') and e.suggestion:
            print(f"💡 {e.suggestion}")

    except Exception as e:
        print(f"❌ Unexpected error: {type(e).__name__}: {e}")

    # Execute fallback if provided
    if fallback_action:
        print("\n🔄 Executing fallback action...")
        return fallback_action()

    return None

# Test the error handling
print("Testing query with error handling:")
safe_query("Generate a simple test dataset with 5 rows")

## 3. Handling Missing Data Gracefully

In [ ]:
# The lui interface returns None/empty instead of raising exceptions
print("Demonstrating graceful missing data handling:\n")

# Before any queries
print(f"lui.df before queries: {lui.df}")
print(f"lui.dfs before queries: {lui.dfs}")
print(f"lui.text before queries: {lui.text}")
print(f"lui.elements before queries: {lui.elements}")

# Make a text-only query
lui("What is the weather like today? Just describe it, no data.")

print("\nAfter text-only query:")
print(f"lui.df: {lui.df}")  # None - no dataframe
print(f"lui.text: {lui.text[:50]}...")  # Has text

# Safe data access patterns
def get_latest_data():
    """Get latest dataframe with fallback."""
    df = lui.df
    if df is None:
        print("No dataframe available, using empty DataFrame")
        return pd.DataFrame()
    return df

# This always works, never raises exception
data = get_latest_data()
print(f"\nData shape: {data.shape}")

## 4. Error Recovery Patterns

In [ ]:
# Pattern 1: Retry with modifications
def query_with_retry(prompt, max_retries=3):
    """Query with automatic retry on failure."""
    for attempt in range(max_retries):
        try:
            response = lui(prompt)
            if not lui.has_errors:
                return response

            # If errors, modify prompt and retry
            print(f"Attempt {attempt + 1} had errors, retrying...")
            prompt = f"Please try again: {prompt}"

        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Attempt {attempt + 1} failed: {e}")
                continue
            else:
                raise

    return None

# Pattern 2: Fallback to simpler query
def query_with_fallback(complex_prompt, simple_prompt):
    """Try complex query, fall back to simple if needed."""
    try:
        print("Trying complex query...")
        response = lui(complex_prompt)
        if lui.df is not None:
            return response
    except Exception:
        pass

    print("Falling back to simple query...")
    return lui(simple_prompt)

# Example usage
query_with_fallback(
    complex_prompt="Create a complex financial model with projections",
    simple_prompt="Create a simple table with 3 columns and 5 rows of sample data"
)

if lui.df is not None:
    print(f"\nGot data with shape: {lui.df.shape}")
    lui.df.head()

## 5. Debugging and Error Inspection

In [ ]:
# Enable traces for debugging
print("Enabling traces for detailed debugging...")
lui.traces = True

# Make a query that might have issues
lui("Try to parse this invalid JSON: {name: 'test, value: 42}")

# Inspect errors if any
if lui.has_errors:
    print("\n🔍 Error Details:")
    for i, error in enumerate(lui.errors, 1):
        print(f"\nError {i}:")
        print(f"  Message: {error.get('message')}")
        print(f"  Type: {error.get('error_type')}")
        if error.get('traceback'):
            print(f"  Traceback: {error.get('traceback')[:200]}...")

# Turn traces back off
lui.traces = False
print("\n✅ Traces disabled")

## 6. History-Based Recovery

In [ ]:
# Use history to recover from errors
def get_last_valid_dataframe():
    """Search history for last valid dataframe."""
    # Check current response first
    if lui.df is not None:
        return lui.df

    # Search history
    print("Current response has no dataframe, searching history...")
    for i in range(-1, -10, -1):  # Check last 10 responses
        try:
            historical = lui[i]
            if historical.df is not None:
                print(f"Found dataframe in response {i}")
                return historical.df
        except IndexError:
            break

    print("No dataframe found in recent history")
    return pd.DataFrame()

# Make some queries
lui("Create a dataset with product sales")
lui("Now just tell me about the weather")  # No dataframe

# Recover last valid dataframe
df = get_last_valid_dataframe()
print(f"\nRecovered dataframe shape: {df.shape}")

## 7. Batch Processing with Error Handling

In [ ]:
# Process multiple queries with error tracking
queries = [
    "Generate sales data for Q1",
    "Generate sales data for Q2",
    "Generate sales data for Q3",
    "Generate sales data for Q4"
]

results = []
errors = []

for query in queries:
    try:
        print(f"Processing: {query}")
        lui(query)

        if lui.df is not None:
            results.append({
                'query': query,
                'rows': len(lui.df),
                'columns': len(lui.df.columns),
                'data': lui.df
            })
        else:
            errors.append({
                'query': query,
                'error': 'No dataframe returned'
            })

    except Exception as e:
        errors.append({
            'query': query,
            'error': str(e)
        })

# Summary
print("\n📊 Batch Processing Summary:")
print(f"Successful: {len(results)}")
print(f"Failed: {len(errors)}")

if results:
    # Combine successful results
    all_data = pd.concat([r['data'] for r in results], ignore_index=True)
    print(f"\nCombined data shape: {all_data.shape}")

## 8. Custom Error Handlers

In [ ]:
# Create custom error handling decorators
from functools import wraps


def handle_louie_errors(default_return=None):
    """Decorator for handling LouieAI errors gracefully."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except NotebookError as e:
                print(f"⚠️  {func.__name__} failed: {e}")
                return default_return
            except Exception as e:
                print(f"❌ Unexpected error in {func.__name__}: {e}")
                return default_return
        return wrapper
    return decorator

# Use the decorator
@handle_louie_errors(default_return=pd.DataFrame({'error': ['No data available']}))
def analyze_sales(region):
    """Analyze sales for a specific region."""
    lui(f"Show sales data for {region} region")
    if lui.df is None:
        raise NotebookError(f"No data available for {region}")
    return lui.df

# This won't crash even if it fails
result = analyze_sales("North America")
print(f"Result shape: {result.shape}")
result.head()

## Best Practices Summary

### 1. **Always Check for Data**
```python
if lui.df is not None:
    # Process dataframe
else:
    # Handle missing data
```

### 2. **Use History for Recovery**
```python
# Access previous successful responses
last_df = lui[-1].df or lui[-2].df or pd.DataFrame()
```

### 3. **Enable Traces for Debugging**
```python
lui.traces = True  # See AI reasoning
# Debug your issue
lui.traces = False  # Turn off for performance
```

### 4. **Check for Errors in Responses**
```python
if lui.has_errors:
    for error in lui.errors:
        print(error['message'])
```

### 5. **Implement Graceful Fallbacks**
```python
try:
    complex_operation()
except NotebookError:
    simple_fallback()
```